In [2]:
########################################################################
# import default libraries
########################################################################
import os
import sys
import gc
########################################################################


########################################################################
# import additional libraries
########################################################################
import numpy as np
import scipy.stats
# from import
import matplotlib.pyplot as plt
from tqdm import tqdm
try:
    from sklearn.externals import joblib
except:
    import joblib
# original lib
import keras_model_unet_1D
import pandas as pd
import keras
import tensorflow as tf
import librosa as lb
import keras.backend as K
import common as com
from __future__ import division 
import numpy as np
import scipy.signal as sps
import scipy.io.wavfile as wf
from sklearn.cluster import KMeans
import sys
import keras_model_uformer
import cv2
from scipy.signal import butter,sosfilt
from tensorflow.keras.utils import to_categorical
import tensorflow_io as tfio
############

/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.9/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrISt10unique_

In [3]:
########################################################################
# load parameter.yaml
########################################################################
param = com.yaml_load()
########################################################################
saved_weight = os.path.join(param["P_MODELSAVE"], 'dataweights.{epoch:02d}-{val_dense_1_binary_accuracy:.2f}.hdf5')
early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_mask_loss', patience = 10)
# model_unet = keras_model.get_model(input_shape=(1,param["feature"]["n_mels"],param["feature"]["n_frames"]), lr = param["fit"]["lr"])

import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
########################################################################
# get data from the list for file paths
########################################################################
def get_label(file_path):
  # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    parts = tf.strings.split(parts[-1], sep='-')
    return parts[0]

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos

def butter_bandpass_filter(data):
    lowcut = 1550
    highcut = 22050/2 - 5
    fs = 22050
    order = 5
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y

def file_read(clean_path, combined_path, part=0):
    """
    convert file_name to a vector array.

    file_name : str
        target .wav file

    return : numpy.array( numpy.array( float ) )
        vector array
        * dataset.shape = (dataset_size, feature_vector_length)
    """
    # generate clean audio data using tf.io
    clean_contents = tf.io.read_file(clean_path)
    clean, sr = tf.audio.decode_wav(clean_contents)
    # clean = tf.transpose(tf.py_function(butter_bandpass_filter, [tf.transpose(clean)], (tf.float32)))
    clean = tf.unstack(clean, axis=-1, num=4)
    dic = {"1": [0, 1, 2, 3], "2": [2, 1, 0, 3], "3": [2, 3, 0, 1], "4": [0, 3, 2, 1]}

    clean   = tf.stack([clean[dic[part][0]], clean[dic[part][1]],
                        clean[dic[part][2]], clean[dic[part][3]]], axis=-1)

    
    # generate combined audio data using tf.io
    combined_contents = tf.io.read_file(combined_path)
    combined, sr = tf.audio.decode_wav(combined_contents)
    combined = tf.unstack(combined, axis=-1, num=4)
    
    combined   = tf.stack([combined[dic[part][0]], combined[dic[part][1]],
                           combined[dic[part][2]], combined[dic[part][3]]], axis=-1)
    return combined, clean


In [ ]:
type_ = "topm_real"
type_2 = "topm"
combined_loc = f"dev_data_doa/combined_ground_truth_{type_}/train/"
clean_loc_train = f"dev_data_filtering/anomaly_ground_truth_topm/Train/"

df = pd.read_csv(f"dev_data_doa/combined_ground_truth_{type_}/train_combined_extra.csv")
df['anomaly_name'] = clean_loc_train + df['anomaly_name'].astype(str) + ".wav"
df['combined_name'] = combined_loc + df['combined_name'].astype(str) 
#TEST
combined_loc_test = f"dev_data_doa/combined_ground_truth_topm_real/test/"
clean_loc_test = f"dev_data_filtering/anomaly_ground_truth_topm/Test/"


jk = pd.read_csv(f"dev_data_doa/combined_ground_truth_topm_real/val_combined_extra.csv")
jk['anomaly_name'] = clean_loc_test + jk['anomaly_name'].astype(str) + ".wav"
jk['combined_name'] = combined_loc_test + jk['combined_name'].astype(str) 

jk_train = jk.sample(frac = 0.5, random_state=42)
jk_val = jk.drop(jk_train.index)

jk = pd.concat([jk_val], ignore_index=True)
jk_combined_test = jk.pop("combined_name")
jk_clean_test = jk.pop("anomaly_name")

# loop of the base directory
print("\n===========================")

# generate dataset
print("============== DATASET_GENERATOR ==============")
df = pd.concat([df], ignore_index=True)
df_clean_train = df.pop("anomaly_name")
df_combined_train = df.pop("combined_name")

# first_data_real = /tmp/cache
BATCH_SIZE = 32
train_dataset = tf.data.Dataset.from_tensor_slices((df_clean_train, df_combined_train))
train_dataset = train_dataset
print(train_dataset.cardinality())

data_train1 = train_dataset.map(lambda x,y: file_read(x,y, part="1"))
data_train2 = train_dataset.map(lambda x,y: file_read(x,y, part="2"))
data_train3 = train_dataset.map(lambda x,y: file_read(x,y, part="3"))
data_train = data_train1.concatenate(data_train2).concatenate(data_train3).cache("tmp_wave1/cache").shuffle(1000).batch(BATCH_SIZE, drop_remainder=True).prefetch(1)
val_dataset = tf.data.Dataset.from_tensor_slices((jk_clean_test, jk_combined_test))
val_dataset = val_dataset

data_val1 = val_dataset.map(lambda x,y: file_read(x,y, part="1"))
print(data_val1.cardinality())

data_val = data_val1.cache("tmp_wave2/cache").shuffle(100).batch(BATCH_SIZE, drop_remainder=True).prefetch(1)

print(data_train1.cardinality())





============== DATASET_GENERATOR ==============
tf.Tensor(8151, shape=(), dtype=int64)
tf.Tensor(404, shape=(), dtype=int64)
tf.Tensor(8151, shape=(), dtype=int64)


In [ ]:
def modified_SDR_loss(true, pred, eps = 1e-8):
    num = K.sum(true * pred)
    den = K.sqrt(K.sum(true * true)) * K.sqrt(K.sum(pred * pred))
    return -(num / (den +eps))

def mel_band(data):
    bandpass = butter_bandpass_filter(data.numpy(), 1550, 22050/2-5, 22050)
    mel_spec = lb.feature.melspectrogram(y=bandpass, n_fft=2048, hop_length=331, n_mels=128, center=False, fmin=0,
                                                fmax=22050/2)
    return mel_spec

def stft_loss(y_true, y_pred):
    n_fft=1024
    hop_length=256
    power=2.0
    sr=22050
    n=0
    # True mel
    true = tf.zeros([1, 513, 173, 4])
    pred = tf.zeros([1, 513, 173, 4])


    for y_true_ in y_true:
        tf.autograph.experimental.set_loop_options(
            shape_invariants=[(true, tf.TensorShape([None, 513, 173, 4]))]
        )
        y_true_ = tf.transpose(y_true_)
        spectrogram_true_ = tfio.audio.spectrogram(y_true_, nfft=n_fft, window=n_fft, stride=hop_length)
        spectrogram_true_ = tf.transpose(spectrogram_true_)
        spectrogram_true_ = tf.expand_dims(spectrogram_true_, 0)
        true = tf.concat([true, spectrogram_true_], 0)
        
    for y_pred_ in y_pred:
        tf.autograph.experimental.set_loop_options(
            shape_invariants=[(pred, tf.TensorShape([None, 513, 173, 4]))]
        )
        y_pred_ = tf.transpose(y_pred_)
        spectrogram_pred_ = tfio.audio.spectrogram(y_pred_, nfft=n_fft, window=n_fft, stride=hop_length)
        spectrogram_pred_ = tf.transpose(spectrogram_pred_)
        spectrogram_pred_ = tf.expand_dims(spectrogram_pred_, 0)
        
        pred = tf.concat([pred, spectrogram_pred_], 0)
        
    mel = K.square(pred - true)
    loss = tf.reduce_mean(mel)
    return loss


def test(y_true, y_pred):
    n_mels=128
    n_fft=2823
    hop_length=330
    power=2.0
    sr=22050
    # True mel
    y_true = tf.transpose(y_true)
    spectrogram_true = tfio.audio.spectrogram(y_true, nfft=n_fft, window=n_fft, stride=hop_length)
    mel_spectrogram_true = tfio.audio.melscale(spectrogram_true, sr, mels=n_mels, fmin=0, fmax=11025, name=None)
    dbscale_mel_spectrogram_true = tfio.audio.dbscale(mel_spectrogram_true, top_db=80)

    # Pred mel
    y_pred = tf.transpose(y_pred)
    spectrogram_pred = tfio.audio.spectrogram(y_pred, nfft=n_fft, window=n_fft, stride=hop_length)
    mel_spectrogram_pred = tfio.audio.melscale(spectrogram_pred, sr, mels=n_mels, fmin=0, fmax=11025, name=None)
    dbscale_mel_spectrogram_pred = tfio.audio.dbscale(mel_spectrogram_pred, top_db=80)

    true = tf.clip_by_value(dbscale_mel_spectrogram_true, 1e-7, 1e3)
    pred = tf.clip_by_value(dbscale_mel_spectrogram_pred, 1e-7, 1e3)
    # Calculate spectral loss
    result = K.square(pred - true)
    return result

def weighted_SDR_loss(true, pred):
    noisy_speech = true[:,:,4:]
    pred_speech = pred
    true_speech = true[:,:,:4]
    
    def SDR_loss (pred, true, eps = 1e-8):
        num = K.sum(pred * true)
        den = K.sqrt(K.sum(true * true)) * K.sqrt(K.sum(pred * pred))
        return -(num / (den + eps))

    pred_noise = noisy_speech - pred_speech
    true_noise = noisy_speech - true_speech
    alpha      = K.sum(true_speech**2) / (K.sum(true_speech**2) + K.sum(true_noise**2)) 
    sound_SDR = SDR_loss(pred_speech, true_speech)
    noise_SDR = SDR_loss(pred_noise, true_noise)
    return alpha * sound_SDR + (1-alpha) * noise_SDR + custom_mse(true, pred)

def spectral_loss_mel(y_true, y_pred):
    n_mels=128
    n_fft=2048
    hop_length=345
    power=2.0
    sr=22050
    n=0
    # True mel
    true = tf.zeros([1, 128, 128, 2])
    pred = tf.zeros([1, 128, 128, 2])
    pred_speech = y_pred
    true_speech = y_true[:,:,:2]

    for y_true_ in true_speech:
        tf.autograph.experimental.set_loop_options(
            shape_invariants=[(true, tf.TensorShape([None, 128, None, 2]))]
        )
        y_true_ = tf.transpose(y_true_)
        spectrogram_true = tfio.audio.spectrogram(y_true_, nfft=n_fft, window=n_fft, stride=hop_length)
        spectrogram_true_ = tfio.audio.melscale(spectrogram_true, sr, mels=n_mels, fmin=0, fmax=11025, name=None)
        spectrogram_true_ = tf.transpose(spectrogram_true_)
        spectrogram_true_ = tfio.audio.dbscale(spectrogram_true_, top_db=80)
        spectrogram_true_ = tf.expand_dims(spectrogram_true_, 0)
        
        true = tf.concat([true, spectrogram_true_], 0)
        
    for y_pred_ in pred_speech:
        tf.autograph.experimental.set_loop_options(
            shape_invariants=[(pred, tf.TensorShape([None, 128, None, 2]))]
        )
        y_pred_ = tf.transpose(y_pred_)
        spectrogram_pred = tfio.audio.spectrogram(y_pred_, nfft=n_fft, window=n_fft, stride=hop_length)
        spectrogram_pred_ = tfio.audio.melscale(spectrogram_pred, sr, mels=n_mels, fmin=0, fmax=11025, name=None)
        spectrogram_pred_ = tf.transpose(spectrogram_pred_)
        spectrogram_pred_ = tfio.audio.dbscale(spectrogram_pred_, top_db=80)
        spectrogram_pred_ = tf.expand_dims(spectrogram_pred_, 0)
        
        pred = tf.concat([pred, spectrogram_pred_], 0)
        
    # l1 = tf.reduce_mean(K.abs(y_pred - y_true))
    # l1 = weighted_SDR_loss(y_true, y_pred)
    mel = K.square(pred - true)
    l2 = tf.reduce_mean(mel)
    # l3 = stft_loss(y_true, y_pred)
    return l2

import math


def custom_mse(y_true, y_pred):
    
    return K.mean(K.square(y_pred - y_true[:,:,:4]), -1)

def custom_mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true[:,:,:4]), -1)
    
def SDR_metric(y_true, y_pred):
    n_mels=128
    n_fft=2048
    hop_length=345
    power=2.0
    sr=22050
    n=0
    # True mel
    true = tf.zeros([1, 128, 128, 4])
    pred = tf.zeros([1, 128, 128, 4])
    pred_speech = y_pred
    true_speech = y_true[:,:,:4]        
    # l1 = tf.reduce_mean(K.abs(y_pred - y_true))
    l1 = weighted_SDR_loss(y_true, y_pred)
    return l1

@tf.function
def SALSA(audio, fs=22050, n_fft=1024, hop_length=345
          , feature_type = 'salsa_lite'):
    audio_input = audio
    # Compute stft
    n_mics = 4
    n_bins = n_fft // 2 + 1
    fmax = 6050  # Hz
    fmin_doa = 520
    fmax_doa = 6050
    lower_bin = tf.cast(tf.math.floor(fmin_doa * n_fft / tf.cast(fs, tf.float32)), tf.int32)  # 512: 1; 256: 0
    upper_bin = tf.cast(tf.math.floor(fmax_doa * n_fft / tf.cast(fs, tf.float32)), tf.int32)  # 9000Hz: 512: 192, 256: 96
    cutoff_bin = tf.cast(tf.math.floor(fmax * n_fft / tf.cast(fs, tf.float32)), tf.int32)  # 9000 Hz, 512 nfft: cutoff_bin = 192
    c = 343
    delta = 2 * math.pi * fs / (n_fft * c)
    freq_vector = tf.experimental.numpy.arange(n_bins)
    mask = tf.cast(tf.equal(freq_vector, 0), tf.float32)
    freq_vector = tf.cast(freq_vector, tf.float32)+mask
    freq_vector = freq_vector[:, None, None]  # n_bins x 1 x 1
    lower_bin = tf.maximum(1, lower_bin)
    log_specs = tf.zeros([n_fft//2+1,128, 1])
    X = tf.zeros([n_fft//2+1,128, 1], dtype=tf.complex64)
    
    for imic in np.arange(4):
        tf.autograph.experimental.set_loop_options(
            shape_invariants=[(X, tf.TensorShape([None, 128, n_fft//2+1]))]
                )
        tf.autograph.experimental.set_loop_options(
            shape_invariants=[(log_specs, tf.TensorShape([None, 128, n_fft//4+1]))]
                )

        stft = (tf.signal.stft(audio_input[imic, :], frame_length=n_fft, frame_step=hop_length, window_fn=tf.signal.hann_window, pad_end=True))
        X = tf.concat([X, tf.transpose(tf.expand_dims(stft, 0))], -1)

    log_specs = log_specs[:,:,1:]
    X = X[:,:,1:]
    # Compute spatial feature
    phase_vector = tf.math.angle(X[:, :, 1:] * tf.math.conj(X[:, :, 0, None])+tf.complex(0., 1e-7))
    
#     # phase_vector = phase_vector / math.pi
    phase_vector = phase_vector / (delta * freq_vector)
    phase_vector = tf.transpose(phase_vector, [2, 1, 0])
    X = tf.transpose(tf.abs(X), [2, 1, 0])
# #     # Crop frequency
    log_specs = X[:, :, lower_bin:cutoff_bin]
    phase_vector = phase_vector[:, :, lower_bin:cutoff_bin]
    phase_vector = phase_vector
    
#     # Stack features
    audio_feature = tf.concat([log_specs, phase_vector], axis=0)
    return audio_feature

def SALSA_loss(y_true, y_pred):
    true = tf.zeros([1, 256, 128, 7])
    pred = tf.zeros([1, 256, 128, 7])
    pred_speech = y_pred
    true_speech = y_true[:,:,:4]

    
    for y_true_ in true_speech:
        tf.autograph.experimental.set_loop_options(
            shape_invariants=[(true, tf.TensorShape([None, 256, 128, None]))]
        )
        y_true_ = tf.transpose(y_true_)
        salsa_true_ = SALSA(y_true_)
        spectrogram_true_ = tf.transpose(salsa_true_)
        spectrogram_true_ = tf.expand_dims(spectrogram_true_, 0)
        true = tf.concat([true, spectrogram_true_], 0)

        
    for y_pred_ in pred_speech:
        tf.autograph.experimental.set_loop_options(
            shape_invariants=[(pred, tf.TensorShape([None, 256, 128, None]))]
        )
        y_pred_ = tf.transpose(y_pred_)
        salsa_pred_ = SALSA(y_pred_)
        spectrogram_pred_ = tf.transpose(salsa_pred_)
        spectrogram_pred_ = tf.expand_dims(spectrogram_pred_, 0)
        
        pred = tf.concat([pred, spectrogram_pred_], 0)

    mel = K.square(pred - true)
    l5 = tf.reduce_mean(mel[:,:,:,:])
    l3 = tf.reduce_mean(K.square(y_pred - y_true[:,:,:4]))
    return l3+l5



In [ ]:
# number of vectors for each wave file
# train model

tf.get_logger().setLevel("ERROR")
checkpoint = keras.callbacks.ModelCheckpoint('unet_model_1D/parts/new_conformer_mic_com_top_{epoch:03d}.h5', monitor='val_loss', verbose=1,
                             save_best_only=True, save_weights_only=False,
                             mode='auto', save_frequency=1)
checkpoint2 = keras.callbacks.ModelCheckpoint('unet_model_1D/parts/new_conformer_mic_com_top_{epoch:03d}.h5', period=10) 

import warnings
warnings.filterwarnings("ignore", message="Converting sparse IndexedSlices")
initial_learning_rate = 0.00007
final_learning_rate = 0.000001
learning_rate_decay_factor = (final_learning_rate /initial_learning_rate)**(1/100)
steps_per_epoch = int(694)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=initial_learning_rate,
                decay_steps=steps_per_epoch,
                decay_rate=learning_rate_decay_factor,
                staircase=True)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=455):
        super().__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps
        
    def get_config(self):
        config = {
        'd_model': self.d_model,
        'warmup_steps': self.warmup_steps,
         }
        return config

    def __call__(self, step):
        d_model = tf.cast(self.d_model, tf.float32)
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(d_model) * tf.math.minimum(arg1, arg2)


# print("============== MODEL TRAINING ==============")
model = keras_model_uformer.wave_u_net(num_initial_filters = 64, num_layers = 4, 
               source_names = ["siren"], num_channels = 4, output_filter_size = 1,
               padding = "same", input_size = 44100, context = False, upsampling_type = "direct",
               output_activation = "tanh", output_type = "direct")

learning_rate = CustomSchedule(256.)

# Load Model
model.summary()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss =
              SALSA_loss,
              metrics= [custom_mse, custom_mae])

# Load Checkpoint

history = model.fit(data_train,
                    epochs=300,
                    verbose=1,
                    validation_data = data_val,
                    callbacks = [checkpoint,checkpoint2])



# print("============== END TRAINING ==============")


# # gc.collect()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 raw_input (InputLayer)         [(None, 44100, 4)]   0           []                               
                                                                                                  
 exp_dims_0 (Lambda)            (None, 1, 44100, 4)  0           ['raw_input[0][0]']              
                                                                                                  
 bilinear_interpol_0 (Lambda)   (None, 1, 32768, 4)  0           ['exp_dims_0[0][0]']             
                                                                                                  
 bilinear_interpol_1 (Lambda)   (None, 1, 65536, 4)  0           ['bilinear_interpol_0[0][0]']    
                                                                                              